In [95]:
%%capture
!pip install transformers datasets evaluate torchmetrics

In [96]:
from huggingface_hub import notebook_login

notebook_login()

## закроем

In [ ]:
from datasets import load_dataset

dataset = load_dataset("Katerina-gopher/ru-summary")

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['summ', 'text'],
        num_rows: 121833
    })
    test: Dataset({
        features: ['summ', 'text'],
        num_rows: 17512
    })
    validation: Dataset({
        features: ['summ', 'text'],
        num_rows: 12103
    })
})

In [ ]:
from transformers import AutoTokenizer

checkpoint = "facebook/bart-large-xsum"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summ"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/121833 [00:00<?, ? examples/s]

Map:   0%|          | 0/17512 [00:00<?, ? examples/s]

Map:   0%|          | 0/12103 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="Katerina-gopher/bart-ru-summary")
# data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="facebook/bart-large-xsum")

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
!git lfs install
!git clone https://huggingface.co/Katerina-gopher/bart-ru-summary

Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.
Cloning into 't5-ru-summary'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 20 (delta 4), reused 0 (delta 0), pack-reused 4
Unpacking objects: 100% (20/20), 3.25 KiB | 665.00 KiB/s, done.


In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("bart-ru-summary")
# model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-xsum")

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="bart-ru-summary",
    # output_dir="facebook/bart-large-xsum",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=15,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    # train_dataset=tokenized_dataset["train"],
    # eval_dataset=tokenized_dataset["test"],
    train_dataset=tokenized_dataset["test"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/content/t5-ru-summary is already a clone of https://huggingface.co/Katerina-gopher/t5-ru-summary. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.177200,2.045876,0.000000,0.000000,0.000000,0.000000,19.000000
2,2.159000,2.029339,0.000000,0.000000,0.000000,0.000000,19.000000
3,2.148800,2.016506,0.000000,0.000000,0.000000,0.000000,19.000000


Several commits (2) will be pushed upstream.
Several commits (3) will be pushed upstream.
Several commits (4) will be pushed upstream.
Several commits (5) will be pushed upstream.
Several commits (6) will be pushed upstream.


KeyboardInterrupt: ignored

## 2

In [ ]:
#@title код из нейронки
# Импортируем необходимые библиотеки
import torch
from transformers import BartTokenizer, BartForConditionalGeneration
from datasets import load_dataset
from torchmetrics.text import ROUGEScore

# Загружаем токенизатор и модель
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-xsum')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-xsum')

# Загружаем датасет GEM/wiki_lingua
dataset = load_dataset('gem', 'wiki_lingua', split='train')

# Функция для токенизации и подготовки данных
def preprocess_data(example):
  # Токенизируем исходный текст и резюме
  input_ids = tokenizer.encode(example['document'], max_length=1024, truncation=True)
  labels = tokenizer.encode(example['summary'], max_length=128, truncation=True)
  # Добавляем специальные токены в начало и конец
  input_ids = [tokenizer.bos_token_id] + input_ids + [tokenizer.eos_token_id]
  labels = [tokenizer.bos_token_id] + labels + [tokenizer.eos_token_id]
  # Возвращаем словарь с входами и метками
  return {'input_ids': input_ids, 'labels': labels}

# Применяем функцию к датасету
dataset = dataset.map(preprocess_data)

# Функция для обрезки или дополнения данных до одинаковой длины
def pad_data(batch):
  # Получаем максимальную длину входов и меток в пакете
  max_input_len = max([len(x) for x in batch['input_ids']])
  max_label_len = max([len(x) for x in batch['labels']])
  # Обрезаем или дополняем нулями входы и метки до максимальной длины
  batch['input_ids'] = [x[:max_input_len] + [0] * (max_input_len - len(x)) for x in batch['input_ids']]
  batch['labels'] = [x[:max_label_len] + [-100] * (max_label_len - len(x)) for x in batch['labels']]
  # Возвращаем пакет с обработанными данными
  return batch

# Применяем функцию к датасету
dataset = dataset.map(pad_data)

# Создаем загрузчик данных с размером пакета 8
dataloader = torch.utils.data.DataLoader(dataset, batch_size=8)

# Определяем функцию потерь и оптимизатор
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Определяем метрику rouge с ключами rouge1, rouge2 и rougeL
rouge = ROUGEScore(rouge_keys=['rouge1', 'rouge2', 'rougeL'])

# Определяем количество эпох обучения
epochs = 10

# Цикл обучения
for epoch in range(epochs):
  # Переводим модель в режим обучения
  model.train()
  # Обнуляем суммарную потерю за эпоху
  epoch_loss = 0.0
  # Обнуляем суммарный rouge за эпоху
  epoch_rouge = {'rouge1': {'precision': 0.0, 'recall': 0.0, 'fmeasure': 0.0},
                 'rouge2': {'precision': 0.0, 'recall': 0.0, 'fmeasure': 0.0},
                 'rougeL': {'precision': 0.0, 'recall': 0.0, 'fmeasure': 0.0}}
  # Перебираем пакеты данных
  for batch in dataloader:
    # Получаем входы и метки из пакета
    input_ids = torch.tensor(batch['input_ids'])
    labels = torch.tensor(batch['labels'])
    # Обнуляем градиенты оптимизатора
    optimizer.zero_grad()
    # Прогоняем входы через модель и получаем выходы
    outputs = model(input_ids=input_ids, labels=labels)
    # Вычисляем потерю на пакете
    loss = criterion(outputs.logits.view(-1, outputs.logits.size(-1)), labels.view(-1))
    # Делаем обратное распространение градиентов
    loss.backward()
    # Обновляем параметры модели
    optimizer.step()
    # Прибавляем потерю к суммарной потере за эпоху
    epoch_loss += loss.item()
    # Генерируем резюме из выходов модели
    summaries = tokenizer.batch_decode(torch.argmax(outputs.logits, dim=-1), skip_special_tokens=True)
    # Переводим метки в текстовый формат
    references = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Обновляем метрику rouge на пакете
    rouge.update([summaries], [references])
  # Выводим среднюю потерю за эпоху
  print(f'Epoch {epoch + 1}, Loss: {epoch_loss / len(dataloader)}')
  # Вычисляем и выводим средний rouge за эпоху
  epoch_rouge = rouge.compute()
  print(f'Epoch {epoch + 1}, Rouge: {epoch_rouge}')


## 3

In [97]:
#@title Импортируем необходимые библиотеки
import torch
from torchmetrics.text.rouge import ROUGEScore
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
from datasets import load_dataset
from tqdm import tqdm
import time

In [98]:
#@title Загружаем предобученную модель и токенизатор BART
model = BartForConditionalGeneration.from_pretrained("Katerina-gopher/bart-ru-summary")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-xsum")

In [105]:
#@title Загружаем датасет 
from datasets import load_dataset

dataset = load_dataset("Katerina-gopher/ru-summary")

  0%|          | 0/3 [00:00<?, ?it/s]

In [106]:
dataset

DatasetDict({
    train: Dataset({
        features: ['summ', 'text'],
        num_rows: 121833
    })
    validation: Dataset({
        features: ['summ', 'text'],
        num_rows: 12103
    })
    test: Dataset({
        features: ['summ', 'text'],
        num_rows: 17512
    })
})

In [107]:
ds = dataset
ds = ds.pop('train')
ds

Dataset({
    features: ['summ', 'text'],
    num_rows: 121833
})

In [108]:
dataset

DatasetDict({
    validation: Dataset({
        features: ['summ', 'text'],
        num_rows: 12103
    })
    test: Dataset({
        features: ['summ', 'text'],
        num_rows: 17512
    })
})

In [76]:
#@title Подготавливаем данные для обучения и валидации
def preprocess_data(example):
  # Токенизируем исходный текст и резюме
  input_ids = tokenizer.encode(example['text'], max_length=1024, truncation=True)
  labels = tokenizer.encode(example['summ'], max_length=128, truncation=True)
  # Добавляем специальные токены в начало и конец
  input_ids = [tokenizer.bos_token_id] + input_ids + [tokenizer.eos_token_id]
  labels = [tokenizer.bos_token_id] + labels + [tokenizer.eos_token_id]
  # Возвращаем словарь с входами и метками
  return {'input_ids': input_ids, 'labels': labels}

In [77]:
#@title Применяем функцию к датасету
dataset = dataset.map(preprocess_data)

Map:   0%|          | 0/12103 [00:00<?, ? examples/s]

Map:   0%|          | 0/17512 [00:00<?, ? examples/s]

In [78]:
dataset

DatasetDict({
    validation: Dataset({
        features: ['summ', 'text', 'input_ids', 'labels', 'input_len', 'label_len'],
        num_rows: 12103
    })
    test: Dataset({
        features: ['summ', 'text', 'input_ids', 'labels', 'input_len', 'label_len'],
        num_rows: 17512
    })
})

In [103]:
#@title Функция для обрезки или дополнения данных до одинаковой длины
def pad_data(batch):
  # Получаем максимальную длину входов и меток в пакете
  max_input_len = max([len(x) for x in batch['input_ids']])
  max_label_len = max([len(x) for x in batch['labels']])
  # Обрезаем или дополняем нулями входы и метки до максимальной длины
  batch['input_ids'] = [x[:max_input_len] + [0] * (max_input_len - len(x)) for x in batch['input_ids']]
  batch['labels'] = [x[:max_label_len] + [-100] * (max_label_len - len(x)) for x in batch['labels']]
  # Возвращаем пакет с обработанными данными
  return batch

In [104]:
#@title Применяем функцию к датасету
dataset = dataset.map(pad_data)

Map:   0%|          | 0/12103 [00:00<?, ? examples/s]

KeyError: ignored

In [41]:
dataset

DatasetDict({
    validation: Dataset({
        features: ['summ', 'text', 'input_ids', 'labels', 'input_len', 'label_len'],
        num_rows: 12103
    })
    test: Dataset({
        features: ['summ', 'text', 'input_ids', 'labels', 'input_len', 'label_len'],
        num_rows: 17512
    })
})

In [44]:
#@title Сортируем данные по длине текста для ускорения обучения
dataset = dataset.sort('input_len')


In [45]:
#@title Определяем функцию потерь, оптимизатор и скорость обучения для модели
criterion = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=1)

In [56]:
#@title Определяем функцию для обучения модели на одной эпохе
def train_epoch(model, dataset, batch_size):
  # Переводим модель в режим обучения
  model.train()
  # Создаем генератор батчей
  dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
  # Инициализируем счетчики потерь и количества батчей
  total_loss = 0
  total_batches = 0
  # Проходим по всем батчам
  for batch in dataloader:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # Получаем входные данные и метки из батча
    input_ids = {k: v.to(device) for k, v in batch['input_ids'].items()}
    labels = {k: v.to(device) for k, v in batch['labels'].items()}
    # Обнуляем градиенты оптимизатора
    optimizer.zero_grad()
    # Прямой проход модели: получаем логиты выходных токенов
    outputs = model(input_ids=input_ids, labels=labels)
    logits = outputs.logits
    # Вычисляем потерю на батче
    loss = criterion(logits.view(-1, logits.shape[-1]), labels.view(-1))
    # Обратный проход: вычисляем градиенты параметров модели
    loss.backward()
    # Обновляем параметры модели с помощью оптимизатора
    optimizer.step()
    # Добавляем потерю к общей сумме потерь
    total_loss += loss.item()
    # Увеличиваем счетчик количества батчей
    total_batches += 1
    # Возвращаем среднюю потерю на эпохе
    return total_loss / total_batches

In [57]:
#@title Определяем функцию для валидации модели на тестовых данных
def validate_epoch(model, dataset, batch_size):
  # Переводим модель в режим оценки
  model.eval()
  # Создаем генератор батчей
  dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
  # Инициализируем списки для хранения референтных и гипотетических резюме
  references = []
  hypotheses = []
  # Проходим по всем батчам
  for batch in dataloader:
    # Получаем входные данные и метки из батча
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    input_ids = {k: v.to(device) for k, v in batch['input_ids'].items()}
    labels = {k: v.to(device) for k, v in batch['labels'].items()}
    # Без градиентов: генерируем резюме с помощью модели
    summaries = model.generate(input_ids=input_ids)
    # Декодируем токены в тексты
    summaries = tokenizer.batch_decode(summaries, skip_special_tokens=True)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Добавляем резюме и метки к спискам
    references.extend(labels)
    hypotheses.extend(summaries)
  # Вычисляем метрики rouge1 rougeL rougeRus на тестовых данных
  rouge_scores = rouge.get_scores(hypotheses, references, avg=True)
  # Возвращаем словарь с метриками
  return rouge_scores

In [118]:
rouge = ROUGEScore(rouge_keys=('rouge1', 'rouge2', 'rougeL')) 

In [58]:
#@title Задаем количество эпох и размер батча
num_epochs = 10
batch_size = 4

In [59]:
#@title Задаем лучшее значение метрики rougeRus и имя файла для сохранения модели
best_rougeRus = 0
model_file = 'bart-large-xsum-ru.pt'

In [60]:
#@title Запускаем цикл обучения модели на заданное количество эпох
for epoch in range(num_epochs):
  # Обучаем модель на одной эпохе и получаем среднюю потерю
  train_loss = train_epoch(model, dataset['test'], batch_size)
  # Валидируем модель на тестовых данных и получаем метрики
  rouge_scores = validate_epoch(model, dataset['validation'], batch_size)
  # Выводим результаты на экран
  print(f'Epoch {epoch + 1}')
  print(f'Train loss: {train_loss:.4f}')
  print(f'ROUGE-1: {rouge_scores["rouge-1"]["f"]:.4f}')
  print(f'ROUGE-L: {rouge_scores["rouge-l"]["f"]:.4f}')
  print(f'ROUGE-Rus: {rouge_scores["rouge-rus"]["f"]:.4f}')
  # Обновляем скорость обучения в зависимости от метрики rougeRus
  scheduler.step(rouge_scores['rouge-rus']['f'])
  # Если метрика rougeRus улучшилась, сохраняем модель
  if rouge_scores['rouge-rus']['f'] > best_rougeRus:
    best_rougeRus = rouge_scores['rouge-rus']['f']
    torch.save(model.state_dict(), model_file)
    print('Model saved!')

RuntimeError: ignored

##5

In [ ]:
#@title Определяем метрику rouge с ключами rouge1, rouge2 и rougeL
rouge = ROUGEScore(rouge_keys=('rouge1', 'rouge2', 'rougeL'))

In [ ]:
#@title Определяем количество эпох обучения
epochs = 10

In [ ]:
#@title Цикл обучения
for epoch in range(epochs):
  # Переводим модель в режим обучения
  model.train()
  # Обнуляем суммарную потерю за эпоху
  epoch_loss = 0.0
  # Обнуляем суммарный rouge за эпоху
  epoch_rouge = {'rouge1': {'precision': 0.0, 'recall': 0.0, 'fmeasure': 0.0},
                 'rouge2': {'precision': 0.0, 'recall': 0.0, 'fmeasure': 0.0},
                 'rougeL': {'precision': 0.0, 'recall': 0.0, 'fmeasure': 0.0}}
  # Перебираем пакеты данных
  for input_ids, labels in zip(test_input_ids_dataloader, test_labels_dataloader):
    # Получаем входы и метки из пакета
    # input_ids = batch1
    # labels = batch2
    # Обнуляем градиенты оптимизатора
    optimizer.zero_grad()
    # Прогоняем входы через модель и получаем выходы
    outputs = model(input_ids=input_ids, labels=labels)
    # Вычисляем потерю на пакете
    loss = criterion(outputs.logits.view(-1, outputs.logits.size(-1)), labels.view(-1))
    # Делаем обратное распространение градиентов
    loss.backward()
    # Обновляем параметры модели
    optimizer.step()
    # Прибавляем потерю к суммарной потере за эпоху
    epoch_loss += loss.item()
    # Генерируем резюме из выходов модели
    summaries = tokenizer.batch_decode(torch.argmax(outputs.logits, dim=-1), skip_special_tokens=True)
    # Переводим метки в текстовый формат
    references = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Обновляем метрику rouge на пакете
    rouge.update([summaries], [references])
  # Выводим среднюю потерю за эпоху
  print(f'Epoch {epoch + 1}, Loss: {epoch_loss / len(dataloader)}')
  # Вычисляем и выводим средний rouge за эпоху
  epoch_rouge = rouge.compute()
  print(f'Epoch {epoch + 1}, Rouge: {epoch_rouge}')


##4

In [ ]:
from tqdm import tqdm # from tqdm.auto import tqdm # notebook compatible
import time

for i1 in range(5):
    for i2 in tqdm(range(300), ascii="░▒█"):
        # do something, e.g. sleep
        time.sleep(0.01)
    for i2 in tqdm(range(300), ascii="░▒█"):
        # do something, e.g. sleep
        time.sleep(0.01)

In [109]:
dataset

DatasetDict({
    validation: Dataset({
        features: ['summ', 'text'],
        num_rows: 12103
    })
    test: Dataset({
        features: ['summ', 'text'],
        num_rows: 17512
    })
})

In [110]:
#@title Определяем функцию для обработки датасета
def process_data(data):
  # data - это список кортежей вида (source_text, target_text)
  # source_text - это текст статьи, target_text - это текст суммарии
  # возвращаем список словарей вида {'input_ids': ..., 'attention_mask': ..., 'labels': ...}
  result = []
  for target_text, source_text in data:
    if len(result) == 1000:
      break
    # токенизируем тексты
    input_ids = tokenizer.encode(source_text, return_tensors='pt', max_length=1024)
    labels = tokenizer.encode(target_text, return_tensors='pt', max_length=128)
    # добавляем маску внимания для ненулевых токенов
    attention_mask = input_ids != tokenizer.pad_token_id
    # добавляем словарь в результат
    result.append({'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels})
  return result

In [111]:
#@title Применяем функцию к датасету
test_ds = process_data(dataset['test'])
val_ds = process_data(dataset['validation'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [112]:
import torch
from torch.utils.data import Dataset, DataLoader
train_dataloader = DataLoader(test_ds, batch_size=8, shuffle=True)
print(len(train_dataloader))
test_dataloader = DataLoader(val_ds, batch_size=8, shuffle=True)
print(len(test_dataloader))
# for batch in train_dataloader: # проходим по батчам
#     print(len(batch)) # печатаем размер батча

125
125


In [113]:
#@title Определяем функцию для обучения модели на одном батче данных
def train_on_batch(batch):
  # batch - это словарь вида {'input_ids': ..., 'attention_mask': ..., 'labels': ...}
  # input_ids - это тензор размера (batch_size, max_length_source)
  # attention_mask - это тензор размера (batch_size, max_length_source)
  # labels - это тензор размера (batch_size, max_length_target)
  # возвращаем значение функции потерь на батче
  # переводим модель в режим обучения
  model.train()
  # переносим данные на устройство (cpu или gpu)
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  batch = {k: v.to(device) for k, v in batch.items()}
  # делаем прямой проход модели и получаем логиты
  outputs = model(**batch)
  logits = outputs.logits
  # вычисляем значение функции потерь
  loss = outputs.loss
  # делаем обратный проход и обновляем параметры модели
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()
  # Прибавляем потерю к суммарной потере за эпоху
  # epoch_loss += loss.item()
  # Генерируем резюме из выходов модели
  labels = batch['labels']
  summaries = tokenizer.batch_decode(torch.argmax(outputs.logits, dim=-1), skip_special_tokens=True)
  # Переводим метки в текстовый формат
  references = tokenizer.batch_decode(labels, skip_special_tokens=True)
  # Обновляем метрику rouge на пакете
  return loss, summaries, references

In [114]:
#@title Определяем функцию для оценки модели на одном батче данных
def evaluate_on_batch(batch):
  # batch - это словарь вида {'input_ids': ..., 'attention_mask': ..., 'labels': ...}
  # input_ids - это тензор размера (batch_size, max_length_source)
  # attention_mask - это тензор размера (batch_size, max_length_source)
  # labels - это тензор размера (batch_size, max_length_target)
  # возвращаем список сгенерированных суммарий и значение функции потерь на батче
  # переводим модель в режим оценки
  model.eval()
  # переносим данные на устройство (cpu или gpu)
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  batch = {k: v.to(device) for k, v in batch.items()}
  # делаем прямой проход модели и получаем логиты
  outputs = model(**batch)
  logits = outputs.logits
  # вычисляем значение функции потерь
  loss = outputs.loss
  # генерируем суммарии с помощью жадного декодирования
  summaries_ids = torch.argmax(logits, dim=-1)
  # преобразуем тензоры в строки
  summaries = tokenizer.batch_decode(summaries_ids, skip_special_tokens=True)
  # возвращаем список суммарий и значение функции потерь
  return summaries, loss.item()

In [115]:
#@title Определяем функцию потерь и оптимизатор
criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [116]:
#@title Определяем количество эпох обучения
epochs = 3

In [120]:
# Импортируем модуль pandas
import pandas as pd

print("Начало обучения")
# for epoch in tqdm(range(epochs), ascii="░▒█"):
# Обнуляем суммарный rouge за эпоху
# epoch_rouge_header = ['rouge1.precision','rouge1.recall', 'rouge1.fmeasure',
#                       'rouge2.precision', 'rouge2.recall', 'rouge2.fmeasure',
#                       'rougeL.precision', 'rougeL.recall', 'rougeL.fmeasure']
# df = pd.DataFrame(epoch_rouge, index=[0])
# df.to_csv("epoch_rouge.csv")
for epoch in range(epochs):
  # Обнуляем суммарную потерю за эпоху
  epoch_loss = 0.0
  epoch_loss1 = 0.0
  # Обнуляем суммарный rouge за эпоху
  epoch_rouge = {'rouge1': {'precision': 0.0, 'recall': 0.0, 'fmeasure': 0.0},
                 'rouge2': {'precision': 0.0, 'recall': 0.0, 'fmeasure': 0.0},
                 'rougeL': {'precision': 0.0, 'recall': 0.0, 'fmeasure': 0.0}}
  # Перебираем пакеты данных
  print("Обучения в батче")
  for batch in tqdm(test_ds, ascii="░▒█"):
    # переводим модель в режим обучения
    model.train()
    # переносим данные на устройство (cpu или gpu)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    batch = {k: v.to(device) for k, v in batch.items()}
    # делаем прямой проход модели и получаем логиты
    outputs = model(**batch)
    logits = outputs.logits
    # вычисляем значение функции потерь
    loss = outputs.loss
    # print(f"{i}: {loss.item()}")
    # делаем обратный проход и обновляем параметры модели
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    # Прибавляем потерю к суммарной потере за эпоху
    epoch_loss += loss.item()
    # Генерируем резюме из выходов модели
    labels = batch['labels']
    summaries = tokenizer.batch_decode(torch.argmax(outputs.logits, dim=-1), skip_special_tokens=True)
    # Переводим метки в текстовый формат
    references = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Обновляем метрику rouge на пакете
    rouge.update(summaries, references)
  # Выводим среднюю потерю за эпоху
  print(f'\nEpoch {epoch + 1}, Loss train: {epoch_loss / len(test_ds)}')
  # Вычисляем и выводим средний rouge за эпоху
  epoch_rouge = rouge.compute()
  print(epoch_rouge)
  df = pd.DataFrame(epoch_rouge, index=[epoch + 1])
  # Выводим датафрейм в табличном виде
  print(df)
  # Выводим датафрейм в файл формата CSV
  df.to_csv("epoch_rouge.csv", mode='a', header=False)
  # оценка
  print("Тест в батче")
  for batch in tqdm(val_ds, ascii="░▒█"):
    # переводим модель в режим оценки
    model.eval()
    # переносим данные на устройство (cpu или gpu)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    batch = {k: v.to(device) for k, v in batch.items()}
    # делаем прямой проход модели и получаем логиты
    outputs = model(**batch)
    logits = outputs.logits
    # вычисляем значение функции потерь
    loss = outputs.loss
    # Прибавляем потерю к суммарной потере за эпоху
    epoch_loss1 += loss.item()
  # Выводим среднюю потерю за эпоху
  print(f'\nEpoch {epoch + 1}, Loss test: {epoch_loss1 / len(val_ds)}')

Начало обучения
Обучения в батче


100%|██████████| 1000/1000 [1:04:03<00:00,  3.84s/it]



Epoch 1, Loss train: 1.574690076828003
{'rouge1_fmeasure': tensor(0.0430), 'rouge1_precision': tensor(0.0430), 'rouge1_recall': tensor(0.0430), 'rouge2_fmeasure': tensor(0.), 'rouge2_precision': tensor(0.), 'rouge2_recall': tensor(0.), 'rougeL_fmeasure': tensor(0.0430), 'rougeL_precision': tensor(0.0430), 'rougeL_recall': tensor(0.0430)}
  rouge1_fmeasure rouge1_precision   rouge1_recall rouge2_fmeasure  \
1  tensor(0.0430)   tensor(0.0430)  tensor(0.0430)      tensor(0.)   

  rouge2_precision rouge2_recall rougeL_fmeasure rougeL_precision  \
1       tensor(0.)    tensor(0.)  tensor(0.0430)   tensor(0.0430)   

    rougeL_recall  
1  tensor(0.0430)  
Тест в батче


100%|██████████| 1000/1000 [05:45<00:00,  2.89it/s]



Epoch 1, Loss test: 1.7365068197250366
Обучения в батче


...


Epoch 2, Loss test: 4.637420654296875
Обучения в батче


100%|██████████| 1000/1000 [1:03:32<00:00,  3.81s/it]



Epoch 3, Loss train: 0.868635048687458
{'rouge1_fmeasure': tensor(0.0733), 'rouge1_precision': tensor(0.0733), 'rouge1_recall': tensor(0.0733), 'rouge2_fmeasure': tensor(0.), 'rouge2_precision': tensor(0.), 'rouge2_recall': tensor(0.), 'rougeL_fmeasure': tensor(0.0733), 'rougeL_precision': tensor(0.0733), 'rougeL_recall': tensor(0.0733)}
  rouge1_fmeasure rouge1_precision   rouge1_recall rouge2_fmeasure  \
3  tensor(0.0733)   tensor(0.0733)  tensor(0.0733)      tensor(0.)   

  rouge2_precision rouge2_recall rougeL_fmeasure rougeL_precision  \
3       tensor(0.)    tensor(0.)  tensor(0.0733)   tensor(0.0733)   

    rougeL_recall  
3  tensor(0.0733)  
Тест в батче


100%|██████████| 1000/1000 [05:57<00:00,  2.80it/s]


Epoch 3, Loss test: 2.9824719429016113


j\chhgg

In [121]:
model.save_pretrained("bart-ru-small")